In [2]:
import torch

In [7]:
#dataset 생성 3번의 쪽지시험 점수(x) 를 통한 최종시험(y) 점수 예측
x_train = torch.FloatTensor([[73, 80, 75],
                             [93, 88, 93],
                             [89, 91, 90],
                             [96, 98, 100],
                             [73, 66, 70]])
y_train = torch.FloatTensor([[152], [185], [180], [196], [142]])

# Model 초기화
W = torch.zeros((3, 1), requires_grad=True)
b = torch.zeros(1, requires_grad=True)

# Optimize 설정
optimizer = torch.optim.SGD([W, b], lr=1e-5)

nb_epochs = 20
for epoch in range(nb_epochs + 1):
    
    #H(x) = w1x1 + w2x2 + ... + wnxn으로 갯수가 늘어남에 따라 term이 너무 길어진다. 따라서 torch 함수를 사용
    hypothesis = x_train.matmul(W) + b # or .mm or @
    
    #Cost 계산
    cost = torch.mean((hypothesis - y_train) ** 2)
    
    optimizer.zero_grad()
    cost.backward()
    optimizer.step()
    
    #squeeze()(unsqueeze()) 차원의 원소가 1인 차원을 제거. detach() 미분값을 계산하기 싫은 경우 사용 
    print('Epoch {:4d}/{} hypothesis: {} Cost: {:.6f}'.format(
        epoch, nb_epochs, hypothesis.squeeze().detach(), cost.item()))


Epoch    0/20 hypothesis: tensor([0., 0., 0., 0., 0.]) Cost: 29661.800781
Epoch    1/20 hypothesis: tensor([67.2578, 80.8397, 79.6523, 86.7394, 61.6605]) Cost: 9298.520508
Epoch    2/20 hypothesis: tensor([104.9128, 126.0990, 124.2466, 135.3015,  96.1821]) Cost: 2915.713135
Epoch    3/20 hypothesis: tensor([125.9942, 151.4381, 149.2133, 162.4896, 115.5097]) Cost: 915.040527
Epoch    4/20 hypothesis: tensor([137.7968, 165.6247, 163.1911, 177.7112, 126.3307]) Cost: 287.936005
Epoch    5/20 hypothesis: tensor([144.4044, 173.5674, 171.0168, 186.2332, 132.3891]) Cost: 91.371017
Epoch    6/20 hypothesis: tensor([148.1035, 178.0144, 175.3980, 191.0042, 135.7812]) Cost: 29.758139
Epoch    7/20 hypothesis: tensor([150.1744, 180.5042, 177.8508, 193.6753, 137.6805]) Cost: 10.445305
Epoch    8/20 hypothesis: tensor([151.3336, 181.8983, 179.2240, 195.1707, 138.7440]) Cost: 4.391228
Epoch    9/20 hypothesis: tensor([151.9824, 182.6789, 179.9928, 196.0079, 139.3396]) Cost: 2.493135
Epoch   10/20 hypo

12회 이후로는 Cost값의 변화가 크게 줄어든다. 

* MultivariateLinearRegressionModel class에서 nn 모듈을 상속하고 nn을 쉽게 만들 수 있게 해준다.

import torch.nn as nn

class MultivariateLinearRegressionModel(nn.Module):
    def __init__(self):
        super().__init__()
        self.linear = nn.Linear(3, 1)
        
    def forward(self, x):
        return self.linear(x)
        
hypothesis = model(x_train)

* torch에서 제공하는 cost function
import torch.nn.functional as F
mse_loss(prediection, y_train)


torch에서 제공하는 library를 통해 code를 수정

In [15]:
#library import
import torch.nn as nn
import torch.nn.functional as F

class LinearRegressionModel(nn.Module):
    def __init__(self):
        super().__init__()
        self.linear = nn.Linear(1, 1)
        
    def forward(self, x):
        return self.linear(x)

class MultivariateLinearRegressionModel(nn.Module):
    def __init__(self):
        super().__init__()
        self.linear = nn.Linear(3, 1)
        
    def forward(self, x):
        return self.linear(x)


#dataset 생성 3번의 쪽지시험 점수(x) 를 통한 최종시험(y) 점수 예측
x_train = torch.FloatTensor([[73, 80, 75], [93, 88, 93], [89, 91, 90], [96, 98, 100], [73, 66, 70]])
y_train = torch.FloatTensor([[152], [185], [180], [196], [142]])

# Model 초기화
#W = torch.zeros((3, 1), requires_grad=True)
#b = torch.zeros(1, requires_grad=True)
model = MultivariateLinearRegressionModel()

# Optimize 설정
#optimizer = torch.optim.SGD([W, b], lr=1e-5)
#W, b 대신 model에 MLRM class를 사용했으므로 해당 parameter를 불러오는 함수를 입력한다.
optimizer = torch.optim.SGD(model.parameters(), lr=1e-5)

nb_epochs = 20
for epoch in range(nb_epochs + 1):
    
    #H(x) = w1x1 + w2x2 + ... + wnxn으로 갯수가 늘어남에 따라 term이 너무 길어진다. 따라서 torch 함수를 사용
    #hypothesis = x_train.matmul(W) + b # or .mm or @
    prediction = model(x_train)
    
    #Cost 계산
    #cost = torch.mean((hypothesis - y_train) ** 2)
    cost = F.mse_loss(prediction, y_train)
    
    optimizer.zero_grad()
    cost.backward()
    optimizer.step()
    
    print('Epoch {:4d}/{} hypothesis: {} Cost: {:.6f}'.format(
        epoch, nb_epochs, prediction.squeeze().detach(), cost.item()))

Epoch    0/20 hypothesis: tensor([ 81.6830,  97.8960,  96.5998, 104.7510,  75.0549]) Cost: 6459.039551
Epoch    1/20 hypothesis: tensor([113.0657, 135.6164, 133.7659, 145.2241, 103.8263]) Cost: 2025.528076
Epoch    2/20 hypothesis: tensor([130.6355, 156.7348, 154.5738, 167.8834, 119.9345]) Cost: 635.857361
Epoch    3/20 hypothesis: tensor([140.4720, 168.5584, 166.2233, 180.5695, 128.9530]) Cost: 200.268967
Epoch    4/20 hypothesis: tensor([145.9789, 175.1781, 172.7454, 187.6719, 134.0023]) Cost: 63.734734
Epoch    5/20 hypothesis: tensor([149.0618, 178.8843, 176.3969, 191.6482, 136.8295]) Cost: 20.937912
Epoch    6/20 hypothesis: tensor([150.7876, 180.9594, 178.4411, 193.8744, 138.4124]) Cost: 7.523036
Epoch    7/20 hypothesis: tensor([151.7537, 182.1213, 179.5855, 195.1207, 139.2988]) Cost: 3.317770
Epoch    8/20 hypothesis: tensor([152.2944, 182.7720, 180.2262, 195.8184, 139.7953]) Cost: 1.999129
Epoch    9/20 hypothesis: tensor([152.5969, 183.1364, 180.5848, 196.2090, 140.0734]) Cos